In [2]:
import pandas as pd 
import numpy as np
from app import get_altum_age, get_hannum_age, get_Han2020_age, get_horvath_age, get_horvath_sb_age, get_pheno_age, get_YingCaus_age, get_ZhangEn_age
import re
import pandas as pd
from clocks import han2020_coefs


zhangen_params = pd.read_table('clocks/zhangen.coef', sep=' ')
zhangen_params['feature'] = zhangen_params['probe']
zhangen_params['coefficient'] = zhangen_params['coef']
zhangen_cpgs = zhangen_params['feature'][1:].tolist()

yingCausAge_params = pd.read_csv('clocks/YingCausAge.csv')
yingCausAge_params['feature'] = yingCausAge_params['term']
yingCausAge_params['coefficient'] = yingCausAge_params['estimate']
yingCausAge_cpgs = yingCausAge_params['feature'][1:].tolist()

han2020_cpgs = han2020_coefs.cpgs
cpgs = pd.read_csv("tools/union_out.csv")
cpgs_list = list(set(cpgs.iloc[:, 0].tolist() + yingCausAge_cpgs + zhangen_cpgs + han2020_coefs.cpgs))
print(len(cpgs_list))


2024-07-16 19:19:31.498530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Lasso from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.

22616


In [98]:
#annotation file 
import re
import pandas as pd

# Read the file
file_path = '/Users/yashagarwal/Downloads/GSE134379_series_matrix-1.txt'

# Read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize lists for columns
sample_titles = []
brain_regions = []
ages = []
diagnoses = []

# Regular expressions to match patterns
brain_region_pattern = re.compile(r'brain region:\s*(\w+)')
age_pattern = re.compile(r'age:\s*(\d+)')
diagnosis_pattern = re.compile(r'diagnosis:\s*([\w\s]+)')

# Parsing logic to fill in sample titles, brain regions, ages, and diagnoses
for line in lines:
    if line.startswith('!Sample_title'):
        titles = line.split('\t')
        sample_titles = [title.strip() for title in titles[1:]]  # Skip the first entry since it's the header
        # Initialize the brain regions, ages, and diagnoses lists with the same length as sample_titles
        brain_regions = [''] * len(sample_titles)
        ages = [''] * len(sample_titles)
        diagnoses = [''] * len(sample_titles)
    if line.startswith('!Sample_characteristics_ch1'):
        characteristics = line.split('\t')
        for i, characteristic in enumerate(characteristics[1:], 1):  # Skip the first entry since it's the header
            brain_region_match = brain_region_pattern.search(characteristic)
            age_match = age_pattern.search(characteristic)
            diagnosis_match = diagnosis_pattern.search(characteristic)
            if brain_region_match:
                brain_regions[i-1] = brain_region_match.group(1)  # Adjust index since titles start from 0
            if age_match:
                ages[i-1] = age_match.group(1)  # Adjust index since titles start from 0
            if diagnosis_match:
                diagnoses[i-1] = diagnosis_match.group(1)  # Adjust index since titles start from 0

# Combine the columns into a structured format
organized_data = list(zip(sample_titles, brain_regions, ages, diagnoses))

df = pd.DataFrame(organized_data, columns=['Sample Title', 'Brain Region', 'Age', 'Diagnosis'])

df_anno = df
#print(df_anno)

# Step 1: Remove the first 4 characters from all values in the first column
df.iloc[:, 0] = df.iloc[:, 0].str[:-1]
df_anno.iloc[:, 0] = df_anno.iloc[:, 0].str[5:]
#print(df_anno)

# First, find the indexes for rows that start with "CBL" and "MTG"
cbl_indices = df_anno.index[df_anno.iloc[:, 0].str.startswith("CBL ")].tolist()
mtg_indices = df_anno.index[df_anno.iloc[:, 0].str.startswith("MTG ")].tolist()
#print(cbl_indices)
#print('1')
# Use iloc to select rows and all columns for each group
df_cbl_anno = df_anno.iloc[cbl_indices, :]
df_mtg_anno = df_anno.iloc[mtg_indices, :]

# Optional: Reset index if you want to have a clean index starting from 0
df_cbl_anno.reset_index(drop=True, inplace=True)
df_mtg_anno.reset_index(drop=True, inplace=True)

# Display the resulting DataFrames
#print("DF_CBL_Anno:")
print(df_cbl_anno)
#print("\nDF_MTG_Anno:")
print(df_mtg_anno)

df_cbl_anno = df_cbl_anno
df_mtg_anno = df_mtg_anno

print(df_cbl_anno.columns[0])

          Sample Title Brain Region Age Diagnosis
0    CBL sample 03--34          CBL  88        AD
1    CBL sample 08--45          CBL  78        AD
2    CBL sample 08--59          CBL  89        AD
3    CBL sample 08--01          CBL  90        AD
4    CBL sample 01--45          CBL  87        AD
..                 ...          ...  ..       ...
399  CBL sample 08--04          CBL  94        ND
400  CBL sample 08--40          CBL  76        ND
401  CBL sample 03--47          CBL  95        ND
402  CBL sample 03--41          CBL  89        ND
403  CBL sample 01--51          CBL  88        ND

[404 rows x 4 columns]
          Sample Title Brain Region Age Diagnosis
0    MTG sample 03--04          MTG  87        AD
1    MTG sample 08--89          MTG  81        AD
2    MTG sample 08--22          MTG  79        AD
3    MTG sample 08--05          MTG  89        AD
4    MTG sample 08--10          MTG  87        AD
..                 ...          ...  ..       ...
399  MTG sample 00--34    

In [99]:
# Define the file path

def process_AD_meth_file(file_path): # Initialize lists to store data
    sample_labels = []
    cpgs_data = []

    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # Parse the header line to get sample labels
        header = lines[0].strip().split('\t')
        sample_labels = header[0::2]  # Assuming every second column is a sample label
        #print(sample_labels)
        
        # Parse the remaining lines for CpG data
        for line in lines[1:]:
            data = line.strip().split('\t')
            cpg_id = data[0]
            values = list(map(float, data[1::2]))  # Extract every second value (ignoring p-values)
            
            # Only process if the CpG ID is in the cpgs_list
            if cpg_id in cpgs_list:
                cpgs_data.append([cpg_id] + values)

    # Create the DataFrame
    columns = ['CpG ID'] + sample_labels
    df = pd.DataFrame(cpgs_data, columns=columns)
    df.columns = df.columns.str.replace('Beta value for ', '')
    return df


file_path = '/Users/yashagarwal/Downloads/GSE134379_processedSamples_cblAD-1.txt'



In [ ]:
file_cblAD_path = '/Users/yashagarwal/Downloads/GSE134379_processedSamples_cblAD.txt'
file_cblND_path = '/Users/yashagarwal/Downloads/GSE134379_processedSamples_cblND.txt'
#file_mtgAD = 
#file_mtgND = 
cbl_AD_meth_df = process_AD_meth_file(file_cblAD_path)
cbl_ND_meth_df = process_AD_meth_file(file_cblND_path)

print(cbl_AD_meth_df.shape)
print(cbl_ND_meth_df.shape)


In [46]:
print(cbl_ND_meth_df)
output_path1 = "/Users/yashagarwal/Downloads/cbl_AD_meth_df.csv"
cbl_AD_meth_df.to_csv(output_path1, index=False)
output_path2 = "/Users/yashagarwal/Downloads/cbl_ND_meth_df.csv"
cbl_ND_meth_df.to_csv(output_path2, index=False)

           CpG ID  CBL 08--85  CBL 97--53   CBL 97--02   CBL 99--29  \
0      cg00105470    0.013952     0.015238     0.014444    0.011933   
1      cg00290506    0.014806     0.015136     0.017558    0.013332   
2      cg00476580    0.013389     0.025375     0.017465    0.012021   
3      cg00500789    0.980253     0.980154     0.978848    0.978747   
4      cg00565688    0.008064     0.010067     0.017287    0.009518   
...           ...         ...          ...          ...         ...   
20867  cg27378424    0.049271     0.050546     0.050543    0.036507   
20868  cg27416437    0.048389     0.051454     0.032856    0.041168   
20869  cg27501458    0.018108     0.026064     0.025089    0.023622   
20870  cg27532722    0.137375     0.207977     0.204652    0.156533   
20871  cg27611781    0.067826     0.089314     0.085561    0.065942   

       CBL 92--14   CBL 96--39   CBL 95--03   CBL 06--61   CBL 06--03   ...  \
0         0.014653     0.013878     0.001312     0.014426     0.0144

In [3]:
cbl_AD_meth_df = pd.read_csv("/Users/yashagarwal/Downloads/cbl_AD_meth_df.csv")
cbl_ND_meth_df = pd.read_csv("/Users/yashagarwal/Downloads/cbl_ND_meth_df.csv")

In [4]:
import pandas as pd

# Assuming cbl_AD_meth_df, cbl_ND_meth_df, and df_cbl_anno are defined DataFrames

# Step 1: Sort both DataFrames by the first column
cbl_AD_meth_df_sorted = cbl_AD_meth_df.sort_values(by=cbl_AD_meth_df.columns[0])
cbl_ND_meth_df_sorted = cbl_ND_meth_df.sort_values(by=cbl_ND_meth_df.columns[0])
print(cbl_AD_meth_df_sorted)
print(cbl_ND_meth_df_sorted)

# Step 2: Merge the sorted DataFrames on the first column
merged_df = pd.merge(cbl_AD_meth_df_sorted, cbl_ND_meth_df_sorted, on=cbl_AD_meth_df_sorted.columns[0], suffixes=('_AD', '_ND'))

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df)


df_AD_meth = merged_df
'''# Step 3: Extract sample IDs from df_cbl_anno
sample_ids = df_cbl_anno.iloc[:, 0].tolist()

# Step 4: Rearrange columns in merged_df according to sample_ids
# Ensure the first column (usually sample IDs) is kept intact and in the beginning
# Iterate through sample_ids and add columns from merged_df that match
new_order = [merged_df.columns[0]] + [col for col in merged_df.columns if col != merged_df.columns[0] and col in sample_ids]

# Step 5: Rearrange columns in merged_df according to new_order
rearranged_merged_df = merged_df[new_order]

# Display the rearranged merged DataFrame
print("\nRearranged Merged DataFrame:")
print(rearranged_merged_df)

# Assign the rearranged DataFrame to df_AD_meth if needed
df_AD_meth = rearranged_merged_df'''


               CPG ID  CBL 03--34   CBL 08--45   CBL 08--59   CBL 08--01   \
0          cg00000292     0.633084     0.633459     0.630471     0.599643   
1          cg00002426     0.169558     0.196611     0.138453     0.154098   
2          cg00003994     0.036220     0.022084     0.042179     0.043651   
3          cg00007981     0.032442     0.023610     0.029634     0.025031   
4          cg00008493     0.941149     0.939420     0.939412     0.952323   
...               ...          ...          ...          ...          ...   
20867  ch.17.1184801R     0.103661     0.090654     0.122913     0.094869   
20868  ch.2.30415474F     0.387092     0.419174     0.411264     0.384361   
20869  ch.2.75889792R     0.098619     0.088316     0.079830     0.067721   
20870  ch.4.73355803R     0.024582     0.027385     0.019034     0.025001   
20871    ch.8.353716R     0.035923     0.037793     0.063157     0.048404   

       CBL 01--45   CBL 00--04   CBL 99--41   CBL 99--06   CBL 00--47   ...

KeyError: 'CPG ID'

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df = df_AD_meth
file_list = ['tools/altum_cpgs_list.txt', 'tools/hannum_cpgs_list.txt', 'tools/horvath_cpgs_list.txt', 'tools/pheno_cpgs_list.txt', 'tools/horvath_SB_cpgs_list.txt', 'tools/han_cpgs_list.txt', 'tools/ying_cpgs_list.txt', 'tools/zhang_cpgs_list.txt']
# Create an empty list to store the overlapping values
overlapping_cpgs = []
all_missing_values = []
# Iterate through the files in file_list
for file in file_list:
    count = 0
    with open(file, 'r') as file:
        cpgs_list = file.readlines()

    cpgs_list = [line.strip() for line in cpgs_list]


    
    missing_values = set(cpgs_list) - set(df.iloc[:, 0])
    all_missing_values = all_missing_values + list(missing_values)

all_missing_values = list(set(all_missing_values))
for value in all_missing_values:
    new_row = [value] + [np.nan] * (df.shape[1] - 1)  # Create a new row with value and 0.5 in other columns
    # print(new_row)
    df.loc[len(df)] = new_row  # Add new row to DataFrame

print(df)
labels = df.iloc[:, 0]  # Assuming the first column contains the CPG labels
print(labels)
methylation_data = df.iloc[:, 1:]  # Assuming the methylation data starts from the second column
print(methylation_data)
print("1")
df.fillna(np.nan, inplace=True)

# Impute missing values using IterativeImputer
imputer_iterative = IterativeImputer(max_iter=10, random_state=0)  # You can adjust max_iter as needed
imputed_data_iterative = imputer_iterative.fit_transform(methylation_data)

# Convert imputed data back to DataFrame
# Create the DataFrame
imputed_df_iterative = pd.DataFrame(imputed_data_iterative, columns=methylation_data.columns)

# Add the labels as the first column
imputed_df_iterative.insert(0, 'CPG ID', labels)

df = imputed_df_iterative
print(df)



           CpG ID  CBL 03--34   CBL 08--45   CBL 08--59   CBL 08--01   \
0      cg00000292     0.633084     0.633459     0.630471     0.599643   
1      cg00002426     0.169558     0.196611     0.138453     0.154098   
2      cg00003994     0.036220     0.022084     0.042179     0.043651   
3      cg00007981     0.032442     0.023610     0.029634     0.025031   
4      cg00008493     0.941149     0.939420     0.939412     0.952323   
...           ...          ...          ...          ...          ...   
22428  cg11946583          NaN          NaN          NaN          NaN   
22429  cg00461841          NaN          NaN          NaN          NaN   
22430  cg08987989          NaN          NaN          NaN          NaN   
22431  cg07757386          NaN          NaN          NaN          NaN   
22432  cg13228642          NaN          NaN          NaN          NaN   

       CBL 01--45   CBL 00--04   CBL 99--41   CBL 99--06   CBL 00--47   ...  \
0         0.584219     0.669120     0.628688

In [74]:
output_path = "/Users/yashagarwal/Downloads/cbl_AD_ND_final_meth_df.csv"
df.to_csv(output_path, index=False)

In [5]:
df = pd.read_csv("/Users/yashagarwal/Downloads/cbl_AD_ND_final_meth_df.csv")
print(df)

           CPG ID  CBL 03--34   CBL 08--45   CBL 08--59   CBL 08--01   \
0      cg00000292     0.633084     0.633459     0.630471     0.599643   
1      cg00002426     0.169558     0.196611     0.138453     0.154098   
2      cg00003994     0.036220     0.022084     0.042179     0.043651   
3      cg00007981     0.032442     0.023610     0.029634     0.025031   
4      cg00008493     0.941149     0.939420     0.939412     0.952323   
...           ...          ...          ...          ...          ...   
22428  cg11946583     0.295567     0.297686     0.296042     0.295262   
22429  cg00461841     0.295567     0.297686     0.296042     0.295262   
22430  cg08987989     0.295567     0.297686     0.296042     0.295262   
22431  cg07757386     0.295567     0.297686     0.296042     0.295262   
22432  cg13228642     0.295567     0.297686     0.296042     0.295262   

       CBL 01--45   CBL 00--04   CBL 99--41   CBL 99--06   CBL 00--47   ...  \
0         0.584219     0.669120     0.628688

In [95]:
altum_pred = np.expand_dims(np.float32(get_altum_age(df)), axis=0)
hannum_pred = np.expand_dims(np.float32(get_hannum_age(df)), axis=0)
horvath_pred = np.expand_dims(np.float32(get_horvath_age(df)), axis=0)
pheno_pred = np.expand_dims(np.float32(get_pheno_age(df)), axis=0)
horvath_sb_pred = np.expand_dims(np.float32(get_horvath_sb_age(df)), axis=0)
pred_zhangEn = np.expand_dims(np.float32(get_ZhangEn_age(df)), axis=0)
pred_han2020 = np.expand_dims(np.float32(get_Han2020_age(df)), axis=0)
pred_yingCaus = np.expand_dims(np.float32(get_YingCaus_age(df)), axis=0)

# Coefficients and intercept
coefficients = np.array([0.3437195049507392, -0.20633851531241806, -0.5173259165634722, 
                         0.2836010547077375, 0.1857810346592119, -0.02208310995460718, 
                         -0.08526975203798473, 0.5640149630320757])
intercept = 19.275324793823557

# Reshape predictions to (404, 1) for broadcasting
altum_pred = altum_pred.reshape(-1, 1)
pred_han2020 = pred_han2020.reshape(-1, 1)
hannum_pred = hannum_pred.reshape(-1, 1)
horvath_pred = horvath_pred.reshape(-1, 1)
horvath_sb_pred = horvath_sb_pred.reshape(-1, 1)
pheno_pred = pheno_pred.reshape(-1, 1)
pred_yingCaus = pred_yingCaus.reshape(-1, 1)
pred_zhangEn = pred_zhangEn.reshape(-1, 1)

# Calculating the ensemble prediction
ensembleLR_pred = np.round(intercept +
                           (coefficients[0] * altum_pred) +
                           (coefficients[1] * pred_han2020) +
                           (coefficients[2] * hannum_pred) +
                           (coefficients[3] * horvath_pred) +
                           (coefficients[4] * horvath_sb_pred) +
                           (coefficients[5] * pheno_pred) +
                           (coefficients[6] * pred_yingCaus) +
                           (coefficients[7] * pred_zhangEn), 2)

# Ensure ensembleLR_pred has the desired shape if needed
ensembleLR_pred = ensembleLR_pred.flatten()  # Flatten if necessary

#print(*ensembleLR_pred, sep='\n')

#for value in df_cbl_anno.iloc[:, 1]:
    #print(value)


results = np.concatenate(np.mean([altum_pred, hannum_pred, horvath_pred, pheno_pred, horvath_sb_pred, pred_zhangEn, pred_han2020, pred_yingCaus],axis=0), ).T
print(results)
for row in results: 
    print(row)
'''columns = ['EnsembleAge','HannumClock', 'HorvathClock', 'PhenoAge', 'HorvathSkinBlood', 'AltumAge', "ZhangEn", "Han2020", "YingCausal"]
df = pd.DataFrame(data = results, columns = columns, index=df.columns[1:])    
def convert_df(df):
        return df.to_csv(index=True).encode('utf-8')

csv = convert_df(df)'''

/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


13/13 [==============================] - 0s 9ms/step
[57.28718084 50.4171245  54.35597427 50.32027785 57.89868153 47.08550075
 36.62961771 58.62067194 48.06502459 65.31285965 55.81972605 55.17945706
 48.51269588 54.55911246 49.4757974  58.40618838 52.26755171 55.77243671
 54.78711455 55.59743576 51.27620699 58.12717369 57.03359432 58.58673629
 49.32083194 40.37257689 58.28964978 47.69926194 61.52447403 53.65279703
 46.54831973 55.37054754 51.64827605 57.2130607  57.35679236 49.47848768
 57.14592921 58.81095816 47.49237831 55.56047082 51.74078764 55.48387717
 48.4182155  55.14880863 46.99790196 52.95261529 53.66689466 62.25137545
 53.2264884  50.71523004 58.37665913 46.30661954 39.78164118 60.01601168
 47.28640005 50.08972822 57.06070234 49.92980789 51.16930503 50.89517711
 49.75684545 56.92658928 46.19480709 56.99970666 56.45489122 53.0000367
 54.32069506 54.63128215 52.40854628 55.87499067 58.59787215 54.07704192
 50.358624   52.5640377  55.78339764 59.41669637 50.33789412 49.97597228

/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:170: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]


30.235922
25.323349
29.075645
27.522392
28.772285
26.280367
20.189978
28.202211
25.07253
34.35525
28.262562
29.448956
26.455181
27.253775
28.496817
27.961393
29.29345
27.766132
28.298613
30.470125
27.574306
29.184803
30.643599
30.645031
27.45473
22.89204
29.439846
25.750023
29.361326
27.507664
24.833649
29.656334
25.193283
28.978401
28.397768
24.788965
30.799065
31.331558
24.330227
27.980585
29.27899
28.752918
27.10042
28.861631
22.52984
26.689495
27.264854
31.778187
26.892834
27.891026
30.698784
24.47767
22.93102
31.261196
23.821379
27.463419
29.735266
26.449724
26.916805
26.06324
25.107527
29.106276
27.513824
30.450523
30.022171
28.759003
28.76983
29.374882
26.88454
28.882538
29.813555
26.504528
25.961529
28.91882
28.953678
29.807465
29.332848
25.488941
28.196697
25.006845
23.869495
29.199577
27.254288
30.6527
28.263077
23.654217
29.116802
33.1154
27.180264
29.691736
27.707443
24.6689
26.82336
31.152914
27.595829
27.067291
26.909063
25.30003
28.506613
29.779062
27.896343
29.983881
27

'columns = [\'EnsembleAge\',\'HannumClock\', \'HorvathClock\', \'PhenoAge\', \'HorvathSkinBlood\', \'AltumAge\', "ZhangEn", "Han2020", "YingCausal"]\ndf = pd.DataFrame(data = results, columns = columns, index=df.columns[1:])    \ndef convert_df(df):\n        return df.to_csv(index=True).encode(\'utf-8\')\n\ncsv = convert_df(df)'